We already have testbenched the crop extensively - assume it works. 

What we want now is data that, when max-normalized in ap_fixed<16,11>, won't just go to zero.

In [1]:
import os
import copy
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import h5py
from sklearn.preprocessing import StandardScaler
from scipy.optimize import least_squares

np.random.seed(0)


Globals

In [2]:
FP_TOTAL = 16
FP_INT = 10
FP_FRAC = FP_TOTAL-FP_INT-1
LUT_BITS = FP_TOTAL-1# number of bits to index LUT
NUM_ROWS = 2**LUT_BITS

Helper functions

In [3]:
def float_to_fixed_point(value: float, total_bits: int, fractional_bits: int) -> str:
    """
    Convert a float to a fixed-point binary representation.
    
    :param value: Float value to convert.
    :param total_bits: Total number of bits for the representation.
    :param fractional_bits: Number of fractional bits.
    :return: Fixed-point binary string.
    """
    scale = 2 ** fractional_bits
    fixed_point_value = int(round(value * scale))
    
    # Handle two's complement representation for negative numbers
    if fixed_point_value < 0:
        fixed_point_value = (1 << total_bits) + fixed_point_value
    
    return f"{fixed_point_value:0{total_bits}b}"


def fixed_point_to_float(binary: str, fractional_bits: int) -> float:
    """
    Convert a fixed-point binary representation to a float.
    
    :param binary: Fixed-point binary string.
    :param fractional_bits: Number of fractional bits.
    :return: Float value.
    """
    total_bits = len(binary)
    int_value = int(binary, 2)
    
    # Handle two's complement for negative numbers
    if int_value >= (1 << (total_bits - 1)):
        int_value -= (1 << total_bits)
    
    return int_value / (2 ** fractional_bits)

In [4]:
# for val_int in range(1,10): #range(2**FP_TOTAL):
reciprocal_LUT = {}
for val_int in range(1,2**(LUT_BITS)):

    val_bin = float_to_fixed_point(val_int, LUT_BITS, 0)

    val_float = fixed_point_to_float(val_bin + "0"*(FP_TOTAL-LUT_BITS), FP_FRAC)
    val_reciprocal_float = 1/val_float
    val_reciprocal_fixed = float_to_fixed_point(val_reciprocal_float, FP_TOTAL, FP_FRAC)

    # print(f"val_bin: {val_bin}, val_float: {val_float}, val_reciprocal_float: {val_reciprocal_float}, val_reciprocal_fixed: {val_reciprocal_fixed}")
    reciprocal_LUT[val_bin] = val_reciprocal_fixed


    val_bin = float_to_fixed_point(-val_int, LUT_BITS, 0)

    val_float = fixed_point_to_float(val_bin + "0"*(FP_TOTAL-LUT_BITS), FP_FRAC)
    val_reciprocal_float = 1/val_float
    val_reciprocal_fixed = float_to_fixed_point(val_reciprocal_float, FP_TOTAL, FP_FRAC)

    reciprocal_LUT[val_bin] = val_reciprocal_fixed

In [5]:
rtl_dir = "04_ref_design"
template_fpath = os.path.join(rtl_dir, "division_LUT_template.sv")
with open(template_fpath, "r") as f:
    template = f.read()

In [6]:
module_rtl = template.replace("{FP_TOTAL}", str(FP_TOTAL))
module_rtl = module_rtl.replace("{FP_TOTAL-1}", str(FP_TOTAL-1))
module_rtl = module_rtl.replace("{FP_INT}", str(FP_INT))
module_rtl = module_rtl.replace("{NUM_ROWS-1}", str(NUM_ROWS-1))
module_rtl = module_rtl.replace("{LUT_BITS}", str(LUT_BITS))
module_rtl = module_rtl.replace("{LUT_BITS-1}", str(LUT_BITS-1))
module_rtl_top, module_rtl_bottom = module_rtl.split("// INSERT LUT HERE")
module_rtl_lines = [module_rtl_top]

for k,v in reciprocal_LUT.items():
    if v[0]=="1": continue
    line = f"        {LUT_BITS}'b{k} : reciprocal = {FP_TOTAL}'b{v};\n"
    if v=="0"*FP_TOTAL:
        line = f"        default : reciprocal = {FP_TOTAL}'b{v};\n"
        module_rtl_lines.append(line)
        break
        
    module_rtl_lines.append(line)

for k,v in reciprocal_LUT.items():
    if v[0]=="0": continue
    line = f"        {LUT_BITS}'b{k} : reciprocal = {v};\n"
    module_rtl_lines.append(line)

module_rtl_lines.append(module_rtl_bottom)

module_rtl = "".join(module_rtl_lines)

In [7]:
module_fpath = os.path.join(rtl_dir, f"division_LUT_ap_fixed_{FP_TOTAL}_{FP_INT}_s{LUT_BITS}.sv")
with open(module_fpath, "w") as f:
    f.write(module_rtl)

# Debugging

### 1. Check if computing max-reciprocal correctly

In [8]:
cf_max_value_bin = "0000001001110011"
cf_max_value_int = fixed_point_to_float(cf_max_value_bin, 0)
cf_max_value_fixed = fixed_point_to_float(cf_max_value_bin, FP_FRAC)
print(f"cf_max_value_int: {cf_max_value_int}")
print(f"cf_max_value_fixed: {cf_max_value_fixed}")
print("")

rec_int = 1/cf_max_value_int
rec_fixed = 1/cf_max_value_fixed
print(f"rec_int: {rec_int}")
print(f"rec_fixed: {rec_fixed}")
print("")

rec_int_bin = float_to_fixed_point(rec_int, FP_TOTAL, 0)
rec_fixed_bin = float_to_fixed_point(rec_fixed, FP_TOTAL, FP_FRAC)
print(f"rec_int_bin: {rec_int_bin}")
print(f"rec_fixed_bin: {rec_fixed_bin}")

norm_coef_bin = "0000000000000010"
norm_coef_fixed = fixed_point_to_float(norm_coef_bin, FP_FRAC)
print("")
print(f"norm_coef_bin: {norm_coef_bin}")
print(f"norm_coef_fixed: {norm_coef_fixed}")

cf_max_value_int: 627.0
cf_max_value_fixed: 19.59375

rec_int: 0.001594896331738437
rec_fixed: 0.051036682615629984

rec_int_bin: 0000000000000000
rec_fixed_bin: 0000000000000010

norm_coef_bin: 0000000000000010
norm_coef_fixed: 0.0625


Checks out

In [9]:
float_to_fixed_point(307, FP_TOTAL, FP_FRAC)

'0010011001100000'

### 2. Check if we're multiplying correctly

In [10]:
s_axis_tdata = "0000000000001100"
s_axis_tdata_int = fixed_point_to_float(s_axis_tdata, 0)
s_axis_tdata_fixed = fixed_point_to_float(s_axis_tdata, FP_FRAC)
print(f"s_axis_tdata_int: {s_axis_tdata_int}")
print(f"s_axis_tdata_fixed: {s_axis_tdata}")
print("")

mul_out_fr = s_axis_tdata_fixed * norm_coef_fixed
mul_out_fr_bin = float_to_fixed_point(mul_out_fr, FP_TOTAL, FP_FRAC)
print(f"mul_out_fr: {mul_out_fr}")
print(f"mul_out_fr_bin: {mul_out_fr_bin}")

# m_axis_tdata = 

s_axis_tdata_int: 12.0
s_axis_tdata_fixed: 0000000000001100

mul_out_fr: 0.0234375
mul_out_fr_bin: 0000000000000001


In [11]:
den = 627
den_bin = float_to_fixed_point(den, FP_TOTAL, 0)

reciprocal = 1/den
reciprocal_bin = float_to_fixed_point(reciprocal, FP_TOTAL, 0)
reciprocal_int = fixed_point_to_float(reciprocal_bin, 0)

print(den, reciprocal_int)

627 0.0


In [12]:
den_test_values = [5, 7, 11, 17, 31, 41, 43, 47, 61,  627]
den_test_bin_values = [float_to_fixed_point(i, FP_TOTAL, FP_FRAC) for i in den_test_values]

den_test_bin_values[0] = "0000001001110011"
print(f"den_test_bin_values:")
for i in den_test_bin_values: print(i)


den_test_bin_values:
0000001001110011
0000000011100000
0000000101100000
0000001000100000
0000001111100000
0000010100100000
0000010101100000
0000010111100000
0000011110100000
0100111001100000


In [13]:
den_test_int_values = [int(fixed_point_to_float(float_to_fixed_point(i, FP_TOTAL, FP_FRAC),0)) for i in den_test_values]
out_test_int_values = [int(fixed_point_to_float(float_to_fixed_point(1/i, FP_TOTAL, FP_FRAC),0)) for i in den_test_values]

print(f"den_test_int_values: {den_test_int_values}")
print(f"out_test_int_values: {out_test_int_values}")

den_test_int_values: [160, 224, 352, 544, 992, 1312, 1376, 1504, 1952, 20064]
out_test_int_values: [6, 5, 3, 2, 1, 1, 1, 1, 1, 0]


In [14]:
a_test = 67
b_test = 12
c_test = a_test*b_test

print(f"den_test: {float_to_fixed_point(a_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(a_test, FP_TOTAL, FP_FRAC), 0))}")
print(f"out_test: {float_to_fixed_point(b_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(b_test, FP_TOTAL, FP_FRAC), 0))}")
print(f"out_test: {float_to_fixed_point(c_test, FP_TOTAL, FP_FRAC)} := int {int(fixed_point_to_float(float_to_fixed_point(c_test, FP_TOTAL, FP_FRAC), 0))}")



den_test: 0000100001100000 := int 2144
out_test: 0000000110000000 := int 384
out_test: 0110010010000000 := int 25728


In [15]:
a_float = [0, 1, 2, 11, 31, 67]
b_float = [12, 12, 12, 31, 31, 31]
c_float = [i*j for (i,j) in zip(a_float, b_float)]

print(f"a_float: {a_float}")
print(f"b_float: {b_float}")
print(f"c_float: {c_float}")

a_float: [0, 1, 2, 11, 31, 67]
b_float: [12, 12, 12, 31, 31, 31]
c_float: [0, 12, 24, 341, 961, 2077]


In [16]:
a_ap_fixed = [float_to_fixed_point(a, FP_TOTAL, FP_FRAC) for a in a_float]
b_ap_fixed = [float_to_fixed_point(b, FP_TOTAL, FP_FRAC) for b in b_float]
c_ap_fixed = [float_to_fixed_point(c, FP_TOTAL, FP_FRAC) for c in c_float]

print(f"a_ap_fixed:")
for a in a_ap_fixed: print(a)

print("\n\n")
print(f"b_ap_fixed:")
for b in b_ap_fixed: print(b)

print("\n\n")
print(f"c_ap_fixed:")
for c in c_ap_fixed: print(c)


a_ap_fixed:
0000000000000000
0000000000100000
0000000001000000
0000000101100000
0000001111100000
0000100001100000



b_ap_fixed:
0000000110000000
0000000110000000
0000000110000000
0000001111100000
0000001111100000
0000001111100000



c_ap_fixed:
0000000000000000
0000000110000000
0000001100000000
0010101010100000
0111100000100000
10000001110100000


In [17]:
a_ap_fixed_to_int = [int(fixed_point_to_float(a, 0)) for a in a_ap_fixed]
b_ap_fixed_to_int = [int(fixed_point_to_float(b, 0)) for b in b_ap_fixed]
c_ap_fixed_to_int = [int(fixed_point_to_float(c, 0)) for c in c_ap_fixed]

print(f"a_ap_fixed_to_int:")
for a in a_ap_fixed_to_int: print(a) 

print("\n\n")
print(f"b_ap_fixed_to_int:")
for b in b_ap_fixed_to_int: print(b)

print("\n\n")
print(f"c_ap_fixed_to_int:")
for c in c_ap_fixed_to_int: print(c)


a_ap_fixed_to_int:
0
32
64
352
992
2144



b_ap_fixed_to_int:
384
384
384
992
992
992



c_ap_fixed_to_int:
0
384
768
10912
30752
-64608
